In [1]:
%reload_ext autoreload
%autoreload 2

# Imports

In [2]:
from kret_notebook import *  # NOTE import first
from kret_matplotlib.mpl_nb_imports import *
from kret_np_pd.np_pd_nb_imports import *
from kret_polars.polars_nb_imports import *
from kret_rosetta.rosetta_nb_imports import *
from kret_lgbm.lgbm_nb_imports import *
from kret_sklearn.sklearn_nb_imports import *
from kret_torch_utils.torch_nb_imports import *
from kret_lightning.lightning_nb_imports import *
from kret_tqdm.tqdm_nb_imports import *
from kret_type_hints.types_nb_imports import *
from kret_utils.utils_nb_imports import *

# from kret_wandb.wandb_nb_imports import *  # NOTE this is slow to import

[kret_np_pd.np_pd_nb_imports] Imported kret_np_pd.np_pd_nb_imports in 0.0218 seconds
Loaded environment variables from /Users/Akseldkw/coding/kretsinger/.env
[kret_matplotlib.mpl_nb_imports] Imported kret_matplotlib.mpl_nb_imports in 0.1764 seconds
[kret_polars.polars_nb_imports] Imported kret_polars.polars_nb_imports in 0.0878 seconds
[kret_rosetta.rosetta_nb_imports] Imported kret_rosetta.rosetta_nb_imports in 0.0000 seconds
[kret_lgbm.lgbm_nb_imports] Imported kret_lgbm.lgbm_nb_imports in 0.0000 seconds
[kret_sklearn.sklearn_nb_imports] Imported kret_sklearn.sklearn_nb_imports in 0.1408 seconds
[kret_torch_utils.torch_nb_imports] Imported kret_torch_utils.torch_nb_imports in 0.3856 seconds
[kret_lightning.lightning_nb_imports] Imported kret_lightning.lightning_nb_imports in 0.0011 seconds
[kret_tqdm.tqdm_nb_imports] Imported kret_tqdm.tqdm_nb_imports in 0.0000 seconds
[kret_type_hints.types_nb_imports] Imported kret_type_hints.types_nb_imports in 0.0004 seconds
[kret_utils.utils_nb_

# Load Data

In [19]:
df = pd.read_csv(UKS_CONSTANTS.DATA_DIR / "medical_cost.csv")

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Id        1338 non-null   int64  
 1   age       1338 non-null   int64  
 2   sex       1338 non-null   object 
 3   bmi       1338 non-null   float64
 4   children  1338 non-null   int64  
 5   smoker    1338 non-null   object 
 6   region    1338 non-null   object 
 7   charges   1338 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 83.8+ KB


In [21]:
dtt(df)

,Id,age,sex,bmi,children,smoker,region,charges
,int64,int64,object,float64,int64,object,object,float64
62,63,64,male,24.700,1,no,northwest,30166.618
486,487,54,female,21.470,3,no,northwest,12475.351
570,571,31,female,29.100,0,no,southwest,3761.292
800,801,42,female,26.180,1,no,southeast,7046.722
1269,1270,45,male,27.500,3,no,southwest,8615.300


## Cleanup

In [22]:
UKS_NP_PD.data_cleanup(df)

In [23]:
dtt(df)

,Id,age,sex,bmi,children,smoker,region,charges
,category,int64,category,float64,int64,bool,category,float64
31,32,18,female,26.315,0,False,northeast,2198.190
314,315,27,female,31.400,0,True,southwest,34838.873
981,982,34,male,21.375,0,False,northeast,4500.339
1222,1223,50,male,25.300,0,False,southeast,8442.667
1244,1245,18,male,33.330,0,False,southeast,1135.941


In [ ]:
df.drop(["Id"], inplace=True, axis=1)

In [ ]:
features = df.drop("charges", axis=1)
target = df["charges"]

X_train, X_val, Y_train, Y_val = train_test_split(features, target, test_size=0.15, random_state=0)

## Scaler

In [28]:
dtt(features)

,age,sex,bmi,children,smoker,region
,int64,category,float64,int64,bool,category
477,25,male,35.625,0,False,northwest
612,18,female,33.155,0,False,northeast
742,53,male,34.105,0,True,northeast
981,34,male,21.375,0,False,northeast
1153,35,female,35.815,1,False,northwest


In [ ]:
float_cols = ["age", "bmi", "children"]
cat_cols = ["sex", "smoker", "region"]

In [31]:
power_transformer = PowerTransformer(method="yeo-johnson", standardize=True)


column_transform = ColumnTransformer(
    transformers=[
        ("scaler", power_transformer, float_cols),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
    verbose=True,
)

In [32]:
pipeline_x = PipelinePD(steps=[("column_transform", column_transform)])

In [ ]:
X_train = pipeline_x.fit_transform_df(X_train)
X_val = pipeline_x.transform_df(X_val)

[ColumnTransformer] ........ (1 of 2) Processing scaler, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s


## Build LGBM Dataset

In [46]:
ds_params = {
    "feature_name": list(X_train.columns),
    "categorical_feature": cat_cols,
}

In [ ]:
train_data = lgbm.Dataset(X_train, label=Y_train, **ds_params)
test_data = lgbm.Dataset(X_val, label=Y_val, reference=train_data, **ds_params)

In [50]:
# train_data.get_feature_name()

# Implementation

In [ ]:
from kret_lgbm.constants_lgbm import LGBM_Defaults


model = LGBMRegressor(**LGBM_Defaults.LGBM_REGRESSOR_DEFAULTS)

In [ ]:
callbacks = [early_stopping(stopping_rounds=50), log_evaluation(period=100)]

In [ ]:
model.fit(X_train, Y_train, eval_set=[(X_val, Y_val)], callbacks=callbacks)

In [ ]:
# model = lgb.train(
#     params=LGBM_Defaults.LGBM_REGRESSOR_DEFAULTS,  # type:ignore[arg-type]
#     train_set=train_data,
#     valid_sets=[test_data],
#     num_boost_round=1000,
#     callbacks=[early_stopping(stopping_rounds=50), log_evaluation(period=100)],
# )

# NOTE ^ bad performance

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000232 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 321
[LightGBM] [Info] Number of data points in the train set: 1003, number of used features: 6
[LightGBM] [Info] Start training from score 13075.303458
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

# Sandbox